# Jane Street Market Prediction

Reference:  https://www.kaggle.com/c/jane-street-market-prediction

In [ ]:

import pandas as pd
import numpy as np

from matplotlib import pyplot as plt
import seaborn as sb

import tensorflow as tf

import sys

from tensorflow.keras.callbacks import EarlyStopping

%matplotlib inline


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Functions To Get Trainning Data

Due to the size of trainning data, supporting functions are available to create and read a sample of trainning data.

In [ ]:
TRAIN_FILE = '/kaggle/input/jane-street-market-prediction/train.csv'
SAMPLE_TRAINNING_FILE = './train-sample.csv'


# Get all trainning data
def get_trainning_data():
    print("Reading training data...")
    df = pd.read_csv(TRAIN_FILE)
    return df

# Create sample of trainning data
def create_trainning_sample(frac = 0.2):
    df_train = get_trainning_data();
    print("Creating sample file...")
    df_sample = df_train.sample(frac=frac)
    df_sample.to_csv(SAMPLE_TRAINNING_FILE, index=False, header=True)
    
def get_sample_trainning_data():
    print("Reading sample training data...")
    # Read training data
    return pd.read_csv(SAMPLE_TRAINNING_FILE)    

### Create Trainning Data Sample

Run this block to create or recreate the sample trainning data file.

In [ ]:
#create_trainning_sample(frac = 0.02);

# Start Exploration

In [ ]:
# Read only sample data created in previous step above
#df_train = get_sample_trainning_data()

# Read all the data
df_train = get_trainning_data()


In [ ]:
print("Generate action column...")
df_train['action'] = 0
df_train.loc[(df_train['resp'] > 0), 'action'] = 1


In [ ]:
df_train.info()

In [ ]:
# Check for balance
sb.distplot(df_train['action']);

## Sort By Time

In [ ]:
df_train.sort_values(by=['date', 'ts_id'], inplace=True)

## Delete rows

In [ ]:
df_train = df_train[df_train['weight'] != 0]

## Drop unecessary columns

In [ ]:
columns_to_delete = ['date', 'weight', 'resp_1', 'resp_2', 'resp_3', 'resp_4', 'resp', 'ts_id']
for column in columns_to_delete:
    df_train.drop(column, axis=1, inplace=True)
    
feature_count = len(df_train.columns) - 1

## Handle Missing Data

In [ ]:
df_train.fillna(df_train.mean(),inplace=True)


# Reset Index

In [ ]:
df_train = df_train.reset_index(drop=True)
df_train

# Train

In [ ]:
# 20 / 80 split
df_validation, df_train = np.split(df_train, [int(.2*len(df_train))])

df_train = df_train.reset_index(drop=True)
df_validation = df_validation.reset_index(drop=True)

df_train_labels = df_train['action'].copy()
df_train.drop('action', inplace=True, axis=1)

df_validation_labels = df_validation['action'].copy()
df_validation.drop('action', inplace=True, axis=1)

print("Shapes: (train={}, train_labels={}, validation={}, validation_labels={})".format(df_train.shape, df_train_labels.shape, df_validation.shape, df_validation_labels.shape))


In [ ]:

input_length = 10
batch_size = 100

generator_train = tf.keras.preprocessing.sequence.TimeseriesGenerator(
    df_train, 
    df_train_labels,
    length=input_length,
    batch_size=batch_size)

generator_validation = tf.keras.preprocessing.sequence.TimeseriesGenerator(
    df_validation, 
    df_validation_labels,
    length=input_length,
    batch_size=batch_size)

In [ ]:
# some values that may or may not get used as I experiment
BATCH_SIZE=50
EPOCHS = 200
STEPS = len(df_train) / BATCH_SIZE
LEARNING_RATE = 0.001

In [ ]:

model = tf.keras.models.Sequential()

model.add(tf.keras.layers.LSTM(
    feature_count, 
    activation='relu', 
    input_shape=(input_length, feature_count), 
    return_sequences=True))

model.add(tf.keras.layers.Dropout(0.2))

model.add(tf.keras.layers.LSTM(
    feature_count, 
    activation='relu', 
    input_shape=(input_length, feature_count), 
    return_sequences=True))

model.add(tf.keras.layers.Dropout(0.2))

model.add(tf.keras.layers.LSTM(
    feature_count, 
    activation='relu', 
    input_shape=(input_length, feature_count), 
    return_sequences=True))

model.add(tf.keras.layers.Dropout(0.2))

model.add(tf.keras.layers.Dense(1, activation="sigmoid"))

model.compile(loss=tf.keras.losses.Hinge(), #tf.keras.losses.BinaryCrossentropy(), 
                optimizer=tf.optimizers.Adam(learning_rate=LEARNING_RATE),
                metrics=["accuracy"])
model.summary()

In [ ]:
es = EarlyStopping(monitor='val_loss', verbose=1, patience=5)

model.fit(generator_train,
          validation_data = generator_validation,
          epochs = EPOCHS,
          verbose = 1,
          callbacks = [es]
    )  


In [ ]:
train_history = model.history.history

plt.figure(1)
plt.subplot(211)
plt.ylim(top=5)
plt.plot(train_history['loss'])
plt.ylabel('Average Loss Per Epoch')
plt.xlabel('Epoch')
plt.title('Average Loss Per Epoch vs Epoch')

plt.subplot(212)
plt.ylim(top=15)
plt.plot(train_history['val_loss'])
plt.ylabel('Val Loss per Epoch')
plt.xlabel('epoch')
plt.title('Val Loss per Epoch vs Epoch')
plt.show()


In [ ]:
import janestreet
env = janestreet.make_env() # initialize the environment
iter_test = env.iter_test() # an iterator which loops over the test set

In [ ]:
print("Starting predictions...")

for (test_df, sample_prediction_df) in iter_test:
    # drop columns
    weight = test_df['weight'].item()
    columns_to_delete = ['date', 'weight']
    for column in columns_to_delete:
        test_df.drop(column, axis=1, inplace=True)
        
    if weight > 0:  
        test_df.fillna(test_df.mean(),inplace=True)
        prediction = model.predict(np.array(test_df.values).reshape(1,1,130));
        if prediction[0][0][0] > 0.79:
            action = 1
        else:
            action = 0
        sample_prediction_df.action = action
    else:
        sample_prediction_df.action = 0
      
    print("\rPrediciton: (index={}, action={})".format(sample_prediction_df.index.values[0], sample_prediction_df.action.values[0]), end="");
    env.predict(sample_prediction_df)   

print("\nFinished predictions")